In [1]:
%pylab inline
import numexpr as ne
import seaborn as sns
from ipywidgets import interact
import time
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('../../..')

In [4]:
%gui qt

In [110]:
from spiketag.base.Binload import bload
from spiketag.view import wave_view
from spiketag.base import ProbeFactory

# Read from raw data which export from fpga

In [6]:
nCh = 160
fs  = 25e3
bf  = bload(nCh, fs)

In [7]:
bf.load('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/mua.bin')

2017-07-10 10:04:38,080 - spiketag - INFO - #############  load data  ###################
2017-07-10 10:04:38,081 - spiketag - INFO - /Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/mua.bin loaded, it contains: 
2017-07-10 10:04:38,083 - spiketag - INFO - 7361618 * 160 points (4711435520 bytes) 
2017-07-10 10:04:38,084 - spiketag - INFO - 160 channels with sampling rate of 25000.0000 
2017-07-10 10:04:38,086 - spiketag - INFO - 294.465 secs (4.908 mins) of data
2017-07-10 10:04:38,088 - spiketag - INFO - #############################################


In [8]:
def bf_to_mua(x):
    y = x.reshape(-1, 32, 5)
    z = np.swapaxes(y, 1, 2)
    z = z.reshape(-1, 160)
    return z

In [9]:
_data = bf_to_mua(bf.npmm)

In [121]:
spk = np.fromfile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/spk.bin', dtype=int32)

In [122]:
ch = spk.reshape(-1,2)[:,0]
t  = spk.reshape(-1,2)[:,1]

In [123]:
_spks = np.vstack((t,ch))
_spks

array([[      1,      17,      17, ..., 7361613, 7361613, 7361617],
       [     79,      67,      70, ...,       9,      73,      20]], dtype=int32)

In [18]:
wv = wave_view(fs=fs, data=_data, spks=_spks, chs=range(96,160))

In [19]:
wv.show()

# Process data which can be readed by spiketag

In [25]:
_data[:,96:].ravel().tofile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_64_25K_07_10_2017.bin')

In [124]:
offset = 96

In [125]:
_spks = _spks[:,np.where(np.in1d(_spks[1],range(96,161)))[0]]
_spks[1] = _spks[1] - offset
_spks

array([[     25,      26,     303, ..., 7179855, 7179998, 7180087],
       [     58,      36,      51, ...,      25,      59,      56]], dtype=int32)

In [128]:
_spks = np.delete(_spks, np.where(_spks[1] == 59)[0], axis=1)

In [129]:
_spks.T.ravel().tofile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_64_25K_07_10_2017.bin.spk')

# Read data as spiketag

In [84]:
nCh = 64
fs  = 25e3
bf  = bload(nCh, fs)

In [85]:
bf.load('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_64_25K_07_10_2017.bin')

2017-07-10 11:02:32,553 - spiketag - INFO - #############  load data  ###################
2017-07-10 11:02:32,555 - spiketag - INFO - /Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_64_25K_07_10_2017.bin loaded, it contains: 
2017-07-10 11:02:32,556 - spiketag - INFO - 7361618 * 64 points (1884574208 bytes) 
2017-07-10 11:02:32,558 - spiketag - INFO - 64 channels with sampling rate of 25000.0000 
2017-07-10 11:02:32,559 - spiketag - INFO - 294.465 secs (4.908 mins) of data
2017-07-10 11:02:32,560 - spiketag - INFO - #############################################


In [86]:
_data = bf.npmm.reshape(-1, 64)

In [100]:
_spk = np.fromfile('/Users/chiy/Documents/HHMI/Github/spiketag/spiketag_test_data/pcie_64_25K_07_10_2017.bin.spk', dtype=int32)
_spk

array([     25,      58,      26, ...,      59, 7180087,      56], dtype=int32)

In [88]:
wv = wave_view(fs=fs, data=_data, spks=_spk)

In [89]:
wv.show()